In [3]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt

In [77]:
file_name = 'tintina3le.txt'

with open(file_name, 'r') as data:
    entry = data.read()

entry_s = entry.splitlines()
nsats = len(entry_s)//3
    
sat_num = entry_s[1][2:7]  # satellite catalog number

epoch0 = float(entry_s[1][18:32])  # element set epoch (UTC)
n0d = float(entry_s[1][33:43])  # 1st derivative of mean motion w.r.t time
n0dd = float(entry_s[1][44:50])**float(entry_s[1][51:52])  # 2nd derivative of mean motion w.r.t. time
Bstar = float(entry_s[1][53:59])**float(entry_s[1][60:61])  # B* drag term
i0 = float(entry_s[2][8:16])  # orbit inclination (degrees)
i0rad = np.deg2rad(i0)

phi0 = float(entry_s[2][17:25])  # right ascension of ascending node (degrees)
e0 = float(entry_s[2][26:33])  # eccentricity (decimal point assumed)
e0 = e0/10**(np.floor(np.log10(e0))+1)

w0 = float(entry_s[2][34:42])  # argument of perigee (degrees)
w0rad = np.deg2rad(w0)

M0 = float(entry_s[2][43:51])  # mean anomaly (degrees)
n0 = float(entry_s[2][53:63])  # mean motion (revolutions/day)
rev0 = float(entry_s[2][63:68])  # revolution number at epoch

## Nomenclature

$n_0 =$ the SGP type "mean" mean motion at epoch <br>
$e_0 =$ the "mean" eccentricity at epoch <br>
$i_0 =$ the "mean" inclination at epoch <br>
$k_e = \sqrt{GM}$ where G is Newton's universal gravitational constant and $M$ is the mass of the Earth <br>

In [75]:
# constants
CK2 = 5.413080e-4
CK4 = 0.62098875e-6
E6A = 1.0e-6
QOMS2T = 1.88027916e-9
S = 1.01222928
TOTHRD = .66666667
XJ3 = -0.253881e-5
XKE = -0.743669161e-1
XKMPER = 6378.135
XMNPDA = 1440
AE = 1.0
CG = 6.67408e-11  # m3 kg-1 s-2
CM = 5.972e24  # kg
CRE = 6371e3  # m

In [82]:
# SGP4 Constants
ke = np.sqrt(G*M)

a1 = (ke/n0)**(2./3)
delta1 = 3./2*CK2/a1**2*(3*np.cos(i0rad)**2-1)/(1-e0**2)**(3/2)
a0 = a1*(1 - 1./3*delta1 - delta1**2 - 134./81*delta1**3)
delta0 = 3./2*CK2/a0**2*(3*np.cos(i0rad)**2-1)/(1-e0**2)**(3./2)
n0dd = n0/(1+delta0)
a0dd = a0/(1-delta0)

theta = np.cos(i0rad)
xi = 1./(a0dd - S)
Beta0 = (1 - e0**2)**(1./2)

eta = a0dd*e0*xi

C2 = QOMS2T*xi**4*n0dd*(1-eta**2)**(-7./2)* \
     (a0dd*(1 + 3./2*eta**2 + 4*e0*eta + e0 * eta**3) + \
     3./2*CK2*xi/(1-eta**2)*(-1./2 + 3./2*theta**2)*(8 + 24*eta**2 + 3*eta**4))
C1 = Bstar*C2

A3OVK2 = -XJ3/CK2*AE**3

C3 = QOMS2T*xi**5*A3OVK2*n0dd*AE*np.sin(i0rad)/e0
C4 = 2*n0dd*QOMS2T*xi**4*a0dd*Beta0**2*(1-eta**2)**(-7./2)*\
     ((2*eta*(1+e0*eta)+1./2*e0+1./2*eta**3)-\
     2*CK2*xi/(a0dd*(1-eta**2))*\
     (3*(1-3*theta**2)*(1+3./2*eta**2-2*e0*eta-1./2*e0*eta**3)+\
     3./4*(1-theta**2)*(2*eta**2-e0*eta-e0*eta**3)*np.cos(2*w0rad)))
C5 = 2*QOMS2T*xi**4*a0dd*Beta0**2*(1-eta**2)**(-7./2)*(1+11./4*eta*(eta+e0)+e0*eta**3)

D2 = 4*a0dd*xi*C1**2
D3 = 4./3*a0dd*xi**2*(17*a0dd + S)*C1**3
D4 = 2./3*a0dd*xi**3*(221*a0dd + 31*S)*C1**4

In [84]:
# define time in future
t0 = 0  # s
t = 1000  # 

In [89]:
# The secular effects of atmospheric drag and gravitation are included
# through the equations

MDF = M0 + (1 + 3*CK2*(-1 + 3*theta**2)/(2*a0dd**2*Beta0**3) + \
           3*CK2**2*(13 - 78*theta**2 + 137*theta**4)/(16*a0dd**4*Beta0**7))*n0dd*(t-t0)


wDF = w0 + (-3*CK2*(1 - 5*theta**2)/(2*a0dd**2*Beta0**4) + \
           3*CK2**2*(7 - 114*theta**2 + 395*theta**4)/(16*a0dd**4*Beta0**8) + \
           5*CK4*(3 - 36*theta**2 + 49*theta**4)/(4*a0dd**4*Beta0**8))*n0dd*(t-t0)


phiDF = phi0 + (-3*CK2*theta/(a0dd**2*Beta0**4) + \
                3*CK2**2*(4*theta - 19*theta**3)/(2*a0dd**4*Beta0**8) + \
                5*CK4*theta*(3 - 7*theta**2)/(2*a0dd**4*Beta0**8))*n0dd*(t-t0)



deltaw = Bstar*C3*(np.cos(w0rad))*(t-t0)
deltaM = -2./3*QOMS2T*Bstar*xi**4*AE/(e0*eta)*((1+eta*np.cos(MDF))**3 - \
                                              (1 + eta*np.cos(M0))**3)
MP = MDF + deltaw + deltaM
w = wDF - deltaw - deltaM
phi = phiDF - 21./2*(n0dd*CK2*theta)/(a0dd**2*Beta0**2)*C1*(t-t0)**2
e = e0 - Bstar*C4*(t-t0) - Bstar*C5*(np.sin(MP)-np.sin(M0))

dt = t-t0
a = a0dd*(1 - C1*dt - D2*dt**2 - D3*dt**3 - D3*dt**4)**2

IL = MP + w + phi + n0dd*(3./2*C1*dt**2 + (D2 + 2*C1**2)*dt**3 + \
                          1./4*(3*D3 + 12*C1*D2 + 10*C1**3)*dt**4 + \
                          1./5*(3*D4 + 12*C1*D3 + 6*D2**2 + \
                                30*C1**2*D2 + 15*C1**4)*dt**5)

Beta = np.sqrt(1 - e**2)
n = ke/a**(3./2)



/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in sqrt


In [96]:
- Bstar*C4*(t-t0) - Bstar*C5*(np.sin(MP)-np.sin(M0))

-2.365463027777004

In [99]:
np.sin(MP)-np.sin(M0)

0.16218251906872927

In [100]:
w

239.90269927550284

In [101]:
w0

239.9027

In [102]:
w0rad

4.187091999424201

In [103]:
wDF

239.90269999340103